# Objective: 
    
In this notebook, we will build a neural network to classifiy the image based on the object present in the image.

# Dataset Info:
    
MNIST database of handwritten digits
Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.

Source: https://www.kaggle.com/warisali2/mnist-database-of-handwritten-digits

# Advanced techniques for training neural networks:
    
Weight Initialization

Nonlinearity (different Activation functions)

Optimizers(different optimizers)

Batch Normalization

Dropout

### 1. Load Libraries and dataset and do experimentation on dataset

In [3]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist    

(X_train, y_train), (X_test, y_test) = mnist.load_data()

### 2. Data Pre-processing

In [5]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [12]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test) 

In [13]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(60000, 784) (10000, 784) (60000, 10) (10000, 10)


### 3. Basic NN model

#### Naive MLP model without any alterations

In [7]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from keras.layers import Flatten

In [8]:
model = Sequential()
model.add(Dense(units=50, input_shape=(784,)))
model.add(Activation('sigmoid'))
model.add(Dense(units=50))
model.add(Activation('sigmoid'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [17]:
sgd = optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(X_train, y_train, batch_size=2, epochs=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 21s 354us/step - loss: 1.7961 - accuracy: 0.3390
Epoch 2/10
60000/60000 [==============================] - 20s 341us/step - loss: 1.5348 - accuracy: 0.4386
Epoch 3/10
60000/60000 [==============================] - 20s 342us/step - loss: 1.4604 - accuracy: 0.4743
Epoch 4/10
60000/60000 [==============================] - 22s 361us/step - loss: 1.4849 - accuracy: 0.4511
Epoch 5/10
60000/60000 [==============================] - 25s 414us/step - loss: 1.4428 - accuracy: 0.4764
Epoch 6/10
60000/60000 [==============================] - 37s 609us/step - loss: 1.2658 - accuracy: 0.5482
Epoch 7/10
60000/60000 [==============================] - 31s 515us/step - loss: 1.1654 - accuracy: 0.5764
Epoch 8/10
60000/60000 [==============================] - 27s 454us/step - loss: 1.0977 - accuracy: 0.6096
Epoch 9/10
60000/60000 [==============================] - 26s 429us/step - loss: 1.1536 - accuracy: 0.5870
Epoch 10/10
60000/60000 [============

In [20]:
results = model.evaluate(X_test, y_test)
print('Test accuracy: ', results[1])

10000/10000 [==============================] - 0s 16us/step
Test accuracy:  0.6262999773025513


## Now, we will train Neural network using below techniques

### 1. Weight Initialization

Changing weight initialization scheme can significantly improve training of the model by preventing vanishing gradient problem 
up to some degree

Ref: https://keras.io/initializers/

In [19]:
def mlp_model():
    model = Sequential()

    model.add(Dense(50, input_shape=(784,), kernel_initializer='he_normal'))
    model.add(Activation('sigmoid'))

    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(Activation('sigmoid'))
    
    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(Activation('sigmoid'))

    model.add(Dense(50, kernel_initializer='he_normal'))
    model.add(Activation('sigmoid'))

    model.add(Dense(10, kernel_initializer='he_normal'))
    model.add(Activation('softmax'))

    sgd = optimizers.sgd(learning_rate=0.001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [22]:
model = mlp_model()
history = model.fit(X_train, y_train, batch_size=200, epochs=100, verbose=1)

Epoch 1/100
60000/60000 [==============================] - 1s 9us/step - loss: 2.5300 - accuracy: 0.0986
Epoch 2/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.4320 - accuracy: 0.1033
Epoch 3/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.3776 - accuracy: 0.1146
Epoch 4/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.3429 - accuracy: 0.1075
Epoch 5/100
60000/60000 [==============================] - 0s 7us/step - loss: 2.3210 - accuracy: 0.1106
Epoch 6/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.3084 - accuracy: 0.1135
Epoch 7/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.3018 - accuracy: 0.1128
Epoch 8/100
60000/60000 [==============================] - 0s 8us/step - loss: 2.2985 - accuracy: 0.1129
Epoch 9/100
60000/60000 [==============================] - 1s 8us/step - loss: 2.2968 - accuracy: 0.1125
Epoch 10/100
60000/60000 [=============================

In [23]:
results = model.evaluate(X_test, y_test)
print('Test Accuracy: ', results[1])

10000/10000 [==============================] - 0s 17us/step
Test Accuracy:  0.39590001106262207


### 2. Nonlinearity (Activation function)

Sigmoid functions suffer from gradient vanishing problem, making training slower

There are many choices apart from sigmoid and tanh; try many of them!

'relu' (rectified linear unit) is one of the most popular ones

Ref: https://keras.io/activations/

In [24]:
def mlp_model():
    model = Sequential()

    model.add(Dense(50, input_shape=(784,)))
    model.add(Activation('relu'))
    
    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('softmax'))

    sgd = optimizers.sgd(learning_rate=0.001)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [25]:
model = mlp_model()
history = model.fit(X_train,y_train, epochs=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 2s 31us/step - loss: 0.9393 - accuracy: 0.7703
Epoch 2/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.3831 - accuracy: 0.8897
Epoch 3/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.2962 - accuracy: 0.9135
Epoch 4/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.2518 - accuracy: 0.9257
Epoch 5/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.2237 - accuracy: 0.9334
Epoch 6/10
60000/60000 [==============================] - 2s 29us/step - loss: 0.2030 - accuracy: 0.9395
Epoch 7/10
60000/60000 [==============================] - 2s 28us/step - loss: 0.1887 - accuracy: 0.9438
Epoch 8/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.1775 - accuracy: 0.9471
Epoch 9/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.1670 - accuracy: 0.9506
Epoch 10/10
60000/60000 [==============================

In [29]:
results = model.evaluate(X_test, y_test)
print('Test Accuracy: ', results[1])

10000/10000 [==============================] - 0s 14us/step
Test Accuracy:  0.9435999989509583


### 3. Batch Normalization

Batch Normalization, one of the methods to prevent the "internal covariance shift" problem, has proven to be highly effective

Normalize each mini-batch before nonlinearity

Ref: https://keras.io/optimizers/

In [30]:
from keras.layers import BatchNormalization, Dropout

def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, )))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))    
    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))    
    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))    
    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))    
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    sgd = optimizers.SGD(lr = 0.001)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [31]:
model = mlp_model()
history = model.fit(X_train, y_train, epochs=20, verbose=1)

Epoch 1/20
60000/60000 [==============================] - 4s 69us/step - loss: 1.5279 - accuracy: 0.5364
Epoch 2/20
60000/60000 [==============================] - 3s 57us/step - loss: 0.8158 - accuracy: 0.7981
Epoch 3/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.5948 - accuracy: 0.8479
Epoch 4/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.4915 - accuracy: 0.8688
Epoch 5/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.4329 - accuracy: 0.8821
Epoch 6/20
60000/60000 [==============================] - 3s 56us/step - loss: 0.3881 - accuracy: 0.8929
Epoch 7/20
60000/60000 [==============================] - 3s 54us/step - loss: 0.3627 - accuracy: 0.8991
Epoch 8/20
60000/60000 [==============================] - 3s 54us/step - loss: 0.3386 - accuracy: 0.9046
Epoch 9/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.3164 - accuracy: 0.9096
Epoch 10/20
60000/60000 [==============================

Batch normalization layer is usually inserted after dense/convolution and before nonlinearity

In [32]:
results = model.evaluate(X_test, y_test)
print('Test Accuracy: ', results[1])

10000/10000 [==============================] - 0s 27us/step
Test Accuracy:  0.9545999765396118


### 4. Dropout

In [38]:
def mlp_model():
    model = Sequential()
    
    model.add(Dense(50, input_shape = (784, )))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))
    model.add(Dropout(0.2))    

    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))   
    model.add(Dropout(0.2)) 

    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))  
    model.add(Dropout(0.2))

    model.add(Dense(50))
    model.add(BatchNormalization())                    
    model.add(Activation('relu'))    
    model.add(Dropout(0.2))

    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [39]:
model = mlp_model()
history = model.fit(X_train, y_train, epochs=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 6s 102us/step - loss: 0.7165 - accuracy: 0.7837
Epoch 2/10
60000/60000 [==============================] - 5s 91us/step - loss: 0.4166 - accuracy: 0.8815
Epoch 3/10
60000/60000 [==============================] - 5s 84us/step - loss: 0.3525 - accuracy: 0.8992
Epoch 4/10
60000/60000 [==============================] - 6s 92us/step - loss: 0.3146 - accuracy: 0.9105
Epoch 5/10
60000/60000 [==============================] - 5s 90us/step - loss: 0.2910 - accuracy: 0.9172
Epoch 6/10
60000/60000 [==============================] - 5s 87us/step - loss: 0.2781 - accuracy: 0.9201
Epoch 7/10
60000/60000 [==============================] - 5s 85us/step - loss: 0.2611 - accuracy: 0.9262
Epoch 8/10
60000/60000 [==============================] - 6s 94us/step - loss: 0.2560 - accuracy: 0.9277
Epoch 9/10
60000/60000 [==============================] - 5s 90us/step - loss: 0.2445 - accuracy: 0.9299
Epoch 10/10
60000/60000 [=============================

In [40]:
results = model.evaluate(X_test, y_test)
print('Test Accuracy: ', results[1])

10000/10000 [==============================] - 0s 32us/step
Test Accuracy:  0.9689000248908997
